In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import numpy as np
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

class Network(nn.Module):
    def __init__(self, state_space, action_space, seed):
        super(Network, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(state_space, 16),
            nn.ReLU(),
#             nn.Linear(64, 64),
#             nn.ReLU(),
            nn.Linear(16, action_space)
        )

    def forward(self, x):
        return F.softmax(self.model(torch.tensor(x).float()))
    
    
    def act(self, x):
        # https://pytorch.org/docs/stable/distributions.html#score-function
        probabilities = self.forward(x)
        m = Categorical(probabilities)
        action = m.sample()
        log = m.log_prob(action)
        
        return action, log
        

In [24]:
import gym
env = gym.make("CartPole-v1")

state_space = env.observation_space.shape[0]
action_space =env.action_space.n

In [25]:
max_episodes = 2000
max_t = 200
update_every = 4
gamma = 0.99

In [26]:
model = Network(state_space, action_space, 1234)
optimiser = optim.Adam(model.parameters(),lr=1e-2)

scores_deque = deque(maxlen=100)

for episode in range(max_episodes):
    state = env.reset()
    
    policy_log = []
    reward_log = []
    
    for t in range(max_t):
        action, log = model.act(state)
        state, reward, done, info = env.step(action.item())
        
        reward_log.append(reward)
        policy_log.append(log)
        
        if done:
            break
        
    discounts = [gamma ** i for i in range(len(reward_log)+1)]    
    rewards = [discount * reward for (discount, reward) in zip(discounts, reward_log)]
    G = np.array(rewards).sum()
    
    policy_loss = np.array([ -log * G for log in policy_log]).sum()
    
    optimiser.zero_grad()
    policy_loss.backward()
    optimiser.step()
    
    scores_deque.append(np.array(reward_log).sum())
    
    
    if episode % 100 == 0:
        print("Episode: {}\tmean reward: {}".format(episode, np.array(scores_deque).mean()))
        

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Episode: 0	mean reward: 13.0
Episode: 100	mean reward: 25.57
Episode: 200	mean reward: 54.44
Episode: 300	mean reward: 122.11
Episode: 400	mean reward: 88.67
Episode: 500	mean reward: 132.69
Episode: 600	mean reward: 184.41
Episode: 700	mean reward: 137.93
Episode: 800	mean reward: 160.5
Episode: 900	mean reward: 171.7
Episode: 1000	mean reward: 167.2
Episode: 1100	mean reward: 156.49
Episode: 1200	mean reward: 163.01
Episode: 1300	mean reward: 190.78
Episode: 1400	mean reward: 186.7
Episode: 1500	mean reward: 198.88
Episode: 1600	mean reward: 180.19
Episode: 1700	mean reward: 166.89
Episode: 1800	mean reward: 187.99
Episode: 1900	mean reward: 200.0
